### Setup

In [2]:
import os, sys, torch
from transformers import AutoModel, AutoTokenizer
from omegaconf import DictConfig, OmegaConf
from hydra.utils import instantiate

/Users/yonatan/.conda/envs/nlp_final_project/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
sys.path.append("../")
from src.models.model_head import HistoricalTextDatingModel, create_model_head_config
from src.utils import init_tracker, DataLoadAndFilter
import transformers

In [4]:
# 1) Set your checkpoint path here
ckpt_base_path = "../outputs/google-bert/bert-base-multilingual-cased/2025-09-15/23-08-03/"
config_path = ckpt_base_path + ".hydra/config.yaml"
if not os.path.exists(config_path):
    raise FileNotFoundError(f"Config path does not exist: {config_path}")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [10]:
cfg = OmegaConf.load(config_path)

### Load model and data

In [53]:

# Load the model and tokenizer
tokenizer = instantiate(cfg.model.tokenizer)
encoder = transformers.AutoModelForMaskedLM.from_pretrained("google-bert/bert-base-multilingual-cased")

# Load datasets using the data loader
data_loader = DataLoadAndFilter(cfg)
train_dataset, eval_dataset = data_loader.load_datasets(base_path="../")

Some weights of the model checkpoint at google-bert/bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Error loading schema ../data/raw/SefariaData/Sefaria-Export-master/schemas/Sheet.json: Expecting value: line 1 column 1 (char 0)
Loading Royal Society texts: 100%|██████████| 100/100 [00:00<00:00, 25852.47it/s]


### Evaluation
First we try a naive approch to complete a scentence with one token to evaluate one-shot

In [48]:
def predict_masked_word(text: str, tokenizer, model, k=5):
    # Tokenize the input
    inputs = tokenizer(text, return_tensors="pt")
    # Find all mask token positions
    mask_token_indices = (inputs["input_ids"] == tokenizer.mask_token_id).nonzero(as_tuple=True)[1]
    # Get model predictions
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = outputs.logits
    # For each mask token, get top-k predictions
    results = []
    for idx in mask_token_indices:
        mask_token_logits = predictions[0, idx, :]
        top_k_tokens = torch.topk(mask_token_logits, k, dim=0).indices.tolist()
        decoded = [tokenizer.decode([token_id]).strip() for token_id in top_k_tokens]
        print(f"Mask at position {idx.item()}: {decoded}")
        results.append(decoded)
    print("\n")
    return


In [49]:
predict_masked_word(f"This text was written between the years {tokenizer.mask_token} to {tokenizer.mask_token}", tokenizer, encoder)
predict_masked_word(f"Israel declared independence between the years {tokenizer.mask_token} and {tokenizer.mask_token}", tokenizer, encoder)
predict_masked_word(f"The United States Of America declared independence between the years {tokenizer.mask_token} and {tokenizer.mask_token}", tokenizer, encoder)
predict_masked_word(f"Barak Obama was born at {tokenizer.mask_token}", tokenizer, encoder)

Mask at position 8: ['I', '1', '.', '2', '5']
Mask at position 10: ['.', ':', 'the', '-', 'a']


Mask at position 7: ['Israel', '1948', '1', '6', '9']
Mask at position 9: ['Israel', '1948', '1947', '1946', '1967']


Mask at position 11: ['2', '1', 'independence', '1918', '3']
Mask at position 13: ['1918', '1910', 'independence', '1919', '1914']


Mask at position 7: [':', '.', 'Obama', 'Karachi', 'Kabul']




Well the results are bad (except of the Israel report to our suprise).   
Let's try to rank the possible dates

In [54]:
def rank_candidates_by_mask_likelihood(sentence_with_mask, candidates, tokenizer, model):
    """
    Given a sentence with one mask token and a list of candidate words,
    returns the candidates sorted by model likelihood for the mask position.
    """
    # Tokenize input
    inputs = tokenizer(sentence_with_mask, return_tensors="pt")
    mask_token_index = (inputs["input_ids"] == tokenizer.mask_token_id).nonzero(as_tuple=True)[1].item()
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits[0, mask_token_index]
    # Get scores for each candidate
    candidate_scores = []
    for word in candidates:
        token_id = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(word)[0])
        score = logits[token_id].item()
        candidate_scores.append((word, score))
    # Sort by score descending
    return sorted(candidate_scores, key=lambda x: x[1], reverse=True)


In [55]:

sentence = f"This text was written in {tokenizer.mask_token}."
candidates = ["1800", "1900", "2000", "1500", "2025"]
ranked = rank_candidates_by_mask_likelihood(sentence, candidates, tokenizer, encoder)
print(ranked)

[('2000', 4.284444332122803), ('1900', 3.8327479362487793), ('1500', 3.5688230991363525), ('1800', 2.058945655822754), ('2025', 0.636199414730072)]
